In [1]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")


Se crea una funciónpara crear un subset

se crea un subset de entrenamiento con las primeras 1000 imagenes de cada categoría
se crea un subset validation de 500 y uno de test de 1000

In [2]:
def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir/subset_name/category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg"
                  for i in range(start_index,end_index)]
        for fname in fnames:
            shutil.copyfile(src = original_dir / fname, dst = dir / fname)

make_subset("train", start_index=0, end_index= 1000)
make_subset("validation", start_index=1000, end_index= 1500)
make_subset("test", start_index=1500, end_index= 2500)


FileExistsError: [WinError 183] No se puede crear un archivo que ya existe: 'cats_vs_dogs_small\\train\\cat'

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape = (180,180,3)) ## Las imagenes son RGB de 180x180 pixeles
x = layers.Rescaling(1./255)(inputs)  ## reescalar la entrada entre 0-1
x = layers.Conv2D(filters = 32, kernel_size = 3, activation = "relu") (x)
x = layers.MaxPooling2D(pool_size = 2) (x)
x = layers.Conv2D(filters = 64, kernel_size = 3, activation = "relu") (x)
x = layers.MaxPooling2D(pool_size = 2) (x)
x = layers.Conv2D(filters = 128, kernel_size = 3, activation = "relu") (x)
x = layers.MaxPooling2D(pool_size = 2) (x)
x = layers.Conv2D(filters = 256, kernel_size = 3, activation = "relu") (x)
x = layers.MaxPooling2D(pool_size = 2) (x)
x = layers.Conv2D(filters = 256, kernel_size = 3, activation = "relu") (x)
x = layers.Flatten() (x)

outputs = layers.Dense(1, activation = "sigmoid") (x)
model = keras.Model(inputs = inputs, outputs = outputs)



In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 89, 89, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 43, 43, 64)        0         
 g2D)                                                        

In [5]:
model.compile(loss="binary_Crossentropy", optimizer = "rmsprop", metrics = ["accuracy"])

Procesamiento de datos

1. leer la imagenes
2. Decodificar de jpg a RGB
3. Convertir la informacion a Tensores de punto flotante
4. Modificar el tamaño de la imagen a 180x180
5. empaquetar en lotes de 32 imagenes

In [8]:

from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(new_base_dir / "train",
                                              image_size = (180, 180),
                                              batch_size = 32)

validation_dataset = image_dataset_from_directory(new_base_dir / "validation",
                                              image_size = (180, 180),
                                              batch_size = 32)

test_dataset = image_dataset_from_directory(new_base_dir / "test",
                                              image_size = (180, 180),
                                              batch_size = 32)

## los datasets son del tamaño (32, 180, 180, 3)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


El objeto datasets es un  iterador, está creado para realizar una eficiente entrada  para los modelos de ML

Normalmente retorna lotes con datos y etiquetas
Se pueden pasar los datasets directamente al metodo fit

funciona asincronamente, mientras procesa nuevos datos los otros están siendo manejados por el modelo

.shuffle(buffer_size) = mezcla elementos dentro de un buffer
.prefetch(buffer_size) = precarga un bufer de elementos en la memoria de GPUpara lograr un mejor rendimiento
.map(callable) = aplica una transformación a cada elemento del conjunto de datos, toma como entrada un unico elemento del dataset
.map() = se utiliza frecuentemente para remodelar la forma del arreglo

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint(filepath="convnet_from_scratch.keras",
                                             save_best_only=True,
                                             monitor="val_loss"
                                             )
]
history = model.fit(train_dataset,
                    epochs=30,
                    validation_data = validation_dataset,
                    callbacks=callbacks)

import matplotlib.pyplot as plt

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)

plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "bo", label="Validation accuracy")
plt.title("train and validation accuracy")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "bo", label = "Training loss")
plt.plot(epochs, val_loss, "b", label= "Validaion loss")
plt.legend()
plt.show

test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset) 
print(f"Test accuracy: {test_acc:.3f}")


Se puede observar que el modelo está sobre ajustado, el accuracy de entrenamiento es 100% mientres el de validación es de 75%
y el de pruebas es de 69%

Se utilizarán tecnicas para evitar el sobre ajuste, el más comun es Data augmentation

se transforman las imagenes existentes y luego se incluyen en los datos de entrnamiento, esto incrementa los datos y el modelo no ve 
dos datos iguales ya que son transformados, esto ayuda al modelo a generalizar mejor las caracteristicas

También se puede usar Dropout

In [ ]:
data_augmentation = keras.Sequential([layers.RandomFlip("horizontal"), ## voltea la imagen un 50% (espejo)
                                      layers.RandomRotation(0.1), ## rota la imagen entre 10% y -10% aleatoreamente 
                                      layers.RandomZoom(0.2), ## incrementa la imagen en un factor de +-20% aleatoreamente
                                      ])



inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="binary_crossentropy",
 optimizer="rmsprop",
 metrics=["accuracy"])

callbacks = [
 keras.callbacks.ModelCheckpoint(
 filepath="convnet_from_scratch_with_augmentation.keras",
 save_best_only=True,
 monitor="val_loss")
]
history = model.fit(
 train_dataset,
 epochs=100,
 validation_data=validation_dataset,
 callbacks=callbacks)

El accuracy ahora varía entre 80% - 85%